Task Divison:

1. Data cleaning by Purnima:  
    - Drop the following columns from both  csv files  using Apache Spark DataFrame API
        - REF_DATE,"UOM","UOM_ID","SCALAR_FACTOR","SCALAR_ID","VECTOR","STATUS","SYMBOL","TERMINATED","DECIMALS" (EXCEPT **VALUE** and **COORDINATE** note that)
    - Remove the records containing the "GEO" column value as "Canada" using Apache Spark DataFrame API
    -  Rename the column name "Business Characteristics" to "Business Types" using Apache Spark DataFrame API
    

    - Create a presenatation slides .... 

2. Data visulization by Purninma(Use Plotly (https://plotly.com/python/) it's really simple (Apache Spark Dataframe can be used):
    -   Already using generative AI by various business types in across canada 
        - Create a pie chart (use value (%) column value)
        - Create a spatial chart showing the use of generative AI across Canada
        - Create a line plot
    - Plans to use generative AI by various business types in each province of canada (use lineplot, heat map , spatial pie chart and Value(%) column)
    - Have not considered using Generative AI(use pie chart and Value(%) column)

1. Data cleaning by Raj 
    - Add a new column "Business classification code" extract the classification code from the Business Characteristics column values for example "Other services (except public administration) [81]" 81 is the business classification code. 
    - 

2. Data visulization by Raj 



Automate tasks to replace employees
Increase automation in tasks, without reducing employment
IT fields (our course)



    


1. Dataset load in the pyspark

In [13]:

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("AnalaysisOfGenerativeAIInCanada").getOrCreate()

# Load the dataset of Business's use of Generative AI in Canada
df = spark.read.csv("../dataset/33100784-eng/33100784.csv", header=True,inferSchema=True)
# df.show(truncate=False)

df_pandas = df.toPandas()

df_pandas.to_csv("GenerativeAIInCanada.csv")


# df.write.format("pdf").save("GenerativeAIInCanada.pdf")

# load the dataset of Value created by Generative AI in Canada across different business sectors
df1 = spark.read.csv("../dataset/33100785-eng/33100785.csv", header=True,inferSchema=True)

df1_pandas = df1.toPandas()
df1_pandas.to_csv("ValueCreatedGenerativeAIInCanada.csv")

df1.show(truncate=False)

# df1.write.format("pdf").save("ValueCreatedGenerativeAIInCanada.pdf")

# Reference https://dash.gallery/dash-clinical-analytics/
# https://www.statcan.gc.ca/o1/en/plus/5847-which-canadian-businesses-are-using-generative-artificial-intelligence-and-why

+--------+------+--------------+---------------------------------------------------------------------+---------------------------------------------------------+-------+------+-------------+---------+-----------+----------+-----+------+------+----------+--------+
|REF_DATE|GEO   |DGUID         |Business characteristics                                             |Value or potential value created by Generative AI        |UOM    |UOM_ID|SCALAR_FACTOR|SCALAR_ID|VECTOR     |COORDINATE|VALUE|STATUS|SYMBOL|TERMINATED|DECIMALS|
+--------+------+--------------+---------------------------------------------------------------------+---------------------------------------------------------+-------+------+-------------+---------+-----------+----------+-----+------+------+----------+--------+
|2023    |Canada|2021A000011124|North American Industry Classification System (NAICS), all industries|Accelerate development of creative content               |Percent|239   |units        |0        |v1579749147|

2. Merge the two datasets by common attribute ie (Coordinate(act as a primary key here))

In [14]:
# Rename 'VALUE' column in df to 'PercentageValueOfUsingGenerativeAI'
df = df.withColumnRenamed("VALUE", "ValueOfUsingGenerativeAI(%)")

# Rename 'VALUE' column in df1 to 'PercentageValueGeneratedByGenerativeAI'
df1 = df1.withColumnRenamed("VALUE", "ValuePrdoucedByGenerativeAI(%)")

df1 = df1.withColumnRenamed("GEO", "GEO1")
df1 = df1.withColumnRenamed("COORDINATE", "COORDINATE1")
df1 = df1.withColumnRenamed("Business characteristics", "Business characteristics1")

joined_df = df.join(df1, df["COORDINATE"] == df1["COORDINATE1"], "inner").drop(df1["COORDINATE1"])
joined_df.show(truncate=False)

24/04/12 14:17:54 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+------+--------------+---------------------------------------------------------------------+---------------------------------------------------------------------+-------+------+-------------+---------+-----------+----------+---------------------------+------+------+----------+--------+--------+------+--------------+---------------------------------------------------------------------+---------------------------------------------------------+-------+------+-------------+---------+-----------+------------------------------+------+------+----------+--------+
|REF_DATE|GEO   |DGUID         |Business characteristics                                             |Business's use of Generative AI                                      |UOM    |UOM_ID|SCALAR_FACTOR|SCALAR_ID|VECTOR     |COORDINATE|ValueOfUsingGenerativeAI(%)|STATUS|SYMBOL|TERMINATED|DECIMALS|REF_DATE|GEO1  |DGUID         |Business characteristics1                                            |Value or potential value create

In [15]:
cleaned_df=joined_df.drop("REF_DATE","GEO1","DGUID", "UOM", "UOM_ID", "SCALAR_FACTOR", "SCALAR_ID", "VECTOR", "STATUS", "SYMBOL", "TERMINATED", "DECIMALS","COORDINATE1","Business characteristics1")

#Drop the rows that have GEO value as Canada in the row
cleaned_df=cleaned_df.filter(cleaned_df.GEO != "Canada")

values_to_exclude = [
    "Made investments outside of Canada",
    "Exported services outside of Canada",
    "Ownership by visible minority, Southeast Asian",
    "Relocated any business or organizational activities or employees from another country into Canada",
    "Exported goods outside of Canada",
    "Ownership by visible minority, South Asian",
    "Imported goods from outside of Canada",
    "Ownership by visible minority, Latin American",
    "North American Industry Classification System (NAICS), all industries",
    "Relocated any business or organizational activities or employees from Canada to another country",
    "Relocated any business or organizational activities or employees from another country to Canada",
    "Imported services from outside of Canada",
    "Age of business or organization, 3 to 10 years old",
    "Majority ownership, woman",
    "Majority ownership, person with a disability",
    "Urban",
    "Age of business or organization, 2 years or less",
    "Majority ownership, immigrant to Canada"
    
]

cleaned_df = cleaned_df.filter(~cleaned_df["Business characteristics"].isin(values_to_exclude))

# cleaned_df.show(truncate=False)
# cleaned_df.write.csv("output.csv", header=True)

#Replacing NULL values with Zero in columns ValuePrdoucedByGenerativeAI(%) and ValueOfUsingGenerativeAI(%)
cleaned_df =  cleaned_df.fillna(0, subset=['ValuePrdoucedByGenerativeAI(%)', 'ValueOfUsingGenerativeAI(%)'])

cleaned_df = cleaned_df.withColumn("Business characteristics", F.regexp_replace(cleaned_df["Business characteristics"], r'\[.*\]', ''))

list_of_orgs = ["100 or more employees","20 to 99 employees","5 to 19 employees","1 to 4 employees"]



cleaned_df = cleaned_df.filter(cleaned_df["Business characteristics"].isin(list_of_orgs))

from pyspark.sql.functions import col


# Find the average ValueOfUsingGenerativeAI(%) of 100 or more employees and 20 to 99 employees   "5 to 19 employees","1 to 4 employees" sort by highest to lowest
from pyspark.sql.functions import col

# Rename the column
cleaned_df = cleaned_df.withColumnRenamed("ValueOfUsingGenerativeAI(%)", "ValueOfUsingGenerativeAI")


from pyspark.sql.functions import col

# Define the values to include
values_to_include = ["Have plans to use Generative AI", "Already using Generative AI"]

# Filter the data
filtered_df = cleaned_df.filter(col("Business's use of Generative AI").isin(values_to_include))

cleaned_df = filtered_df.groupBy("Business characteristics").avg("ValueOfUsingGenerativeAI").sort("avg(ValueOfUsingGenerativeAI)", ascending=False)
# cleaned_df.show(truncate=False)

#create a chart to show the average value of using Generative AI by different business sectors using px
import plotly.express as px

fig = px.bar(cleaned_df, x='Business characteristics', y='avg(ValueOfUsingGenerativeAI)', color='Business characteristics', title='Generative AI  used by various size of organizations in Canada')
fig.update_xaxes(title_text='Types of Organization')
fig.update_yaxes(title_text='% Use of Generative AI')
fig.show()

fig.write_image("../visualization/GenerativeAIUsedByVariousSizeOfOrganizationsInCanada.pdf")


NameError: name 'F' is not defined

3. Data cleaning using Pyspark Dataframe API

In [ ]:
#Drop columns that are not required for analysis for example "DGUID", "UOM", "UOM_ID", "SCALAR_FACTOR", "SCALAR_ID", "VECTOR", "COORDINATE", "STATUS", "SYMBOL", "TERMINATED", "DECIMALS"
cleaned_df=joined_df.drop("REF_DATE","GEO1","DGUID", "UOM", "UOM_ID", "SCALAR_FACTOR", "SCALAR_ID", "VECTOR", "STATUS", "SYMBOL", "TERMINATED", "DECIMALS","COORDINATE1","Business characteristics1")

#Drop the rows that have GEO value as Canada in the row
cleaned_df=cleaned_df.filter(cleaned_df.GEO != "Canada")


values_to_exclude = [
    "Made investments outside of Canada",
    "Exported services outside of Canada",
    "Ownership by visible minority, Southeast Asian",
    "Relocated any business or organizational activities or employees from another country into Canada",
    "100 or more employees",
    "Exported goods outside of Canada",
    "Ownership by visible minority, South Asian",
    "Imported goods from outside of Canada",
    "Ownership by visible minority, Latin American",
    "20 to 99 employees",
    "North American Industry Classification System (NAICS), all industries",
    "Relocated any business or organizational activities or employees from Canada to another country",
    "Relocated any business or organizational activities or employees from another country to Canada",
    "Imported services from outside of Canada",
    "Age of business or organization, 3 to 10 years old",
    "Majority ownership, woman",
    "Majority ownership, person with a disability",
    "Urban",
    "5 to 19 employees",
    "Age of business or organization, 2 years or less",
    "Majority ownership, immigrant to Canada"
    
]

cleaned_df = cleaned_df.filter(~cleaned_df["Business characteristics"].isin(values_to_exclude))

# cleaned_df.show(truncate=False)
# cleaned_df.write.csv("output.csv", header=True)

#Replacing NULL values with Zero in columns ValuePrdoucedByGenerativeAI(%) and ValueOfUsingGenerativeAI(%)
cleaned_df =  cleaned_df.fillna(0, subset=['ValuePrdoucedByGenerativeAI(%)', 'ValueOfUsingGenerativeAI(%)'])
# cleaned_df.show(truncate=False)


from pyspark.sql import functions as F

#Remove [31-33] such values like [23] from Business characteristics column
cleaned_df = cleaned_df.withColumn("Business characteristics", F.regexp_replace(cleaned_df["Business characteristics"], r'\[.*\]', ''))
cleaned_df.show(truncate=False)

# cleaned_df.write.csv("output.csv", header=True) 

print("Total no of records in the dataset => " ,cleaned_df.count())
print("Total records of Ontario province alone =>" ,cleaned_df.filter(cleaned_df["GEO"] == "Ontario").count())


+-------------------------+----------------------------------------------+---------------------------------------------------------------------+----------+---------------------------+---------------------------------------------------------+------------------------------+
|GEO                      |Business characteristics                      |Business's use of Generative AI                                      |COORDINATE|ValueOfUsingGenerativeAI(%)|Value or potential value created by Generative AI        |ValuePrdoucedByGenerativeAI(%)|
+-------------------------+----------------------------------------------+---------------------------------------------------------------------+----------+---------------------------+---------------------------------------------------------+------------------------------+
|Newfoundland and Labrador|Agriculture, forestry, fishing and hunting    |Already using Generative AI                                          |2.2.1     |2.9                       

4. Data visulization

In [ ]:
df = spark.read.csv("output.csv", header=True,inferSchema=True)

df =df.withColumnRenamed("GEO","Province")

from pyspark.sql import functions as F

top5_business_chars = cleaned_df \
                .groupBy("Business characteristics","Business's use of Generative AI") \
                .agg(F.avg("ValueOfUsingGenerativeAI(%)") \
                .alias("AverageValueOfUsingGenerativeAI(%)")) \
                .sort("AverageValueOfUsingGenerativeAI(%)", ascending=False)
                
top5_business_chars = top5_business_chars.filter(top5_business_chars["Business's use of Generative AI"] == "Already using Generative AI") \
            .limit(5)


top5_business_chars.show(truncate=False)

# # Create a bar chart
import plotly.express as px

# Convert Spark DataFrame to Pandas DataFrame
top5_business_chars_pd = top5_business_chars.toPandas()

top5_business_chars_pd = top5_business_chars_pd.rename(columns={'Business characteristics': 'Business Sectors',
                                                                'AverageValueOfUsingGenerativeAI(%)': 'Average Value Of Using Generative AI (%)'})

# Create a bar chart
fig = px.bar(top5_business_chars_pd, 
             x='Business Sectors', 
             y='Average Value Of Using Generative AI (%)', 
             title='Top 5 Business Sectors Already Using Generative AI across Canada',
             labels={'Average Value Of Using Generative AI (%)':'% Value Of Using Generative AI'},
             color='Business Sectors')



fig.show()
fig.write_image("../visualization/Top5BusinessCharacteristicsAlreadyUsingGenerativeAI.pdf")


# Top 5 provinces using Generative AI in Canada

top5_provinces = cleaned_df \
                .groupBy("GEO","Business's use of Generative AI") \
                .agg(F.avg("ValueOfUsingGenerativeAI(%)") \
                .alias("AverageValueOfUsingGenerativeAI(%)")) \
                .sort("AverageValueOfUsingGenerativeAI(%)", ascending=False)

top5_provinces = top5_provinces.filter(top5_provinces["Business's use of Generative AI"] == "Already using Generative AI") \
            .limit(5)   

top5_provinces.show(truncate=False)


# create a bar chart
top5_provinces_pd = top5_provinces.toPandas()

top5_provinces_pd = top5_provinces_pd.rename(columns={'GEO': 'Province',
                                                        'AverageValueOfUsingGenerativeAI(%)': '% Value Of Using Generative AI'})


fig = px.bar(top5_provinces_pd,
                x='Province',
                y='% Value Of Using Generative AI',
                title='Top 5 Provinces Already Using Generative AI in Canada',
                labels={'% Value Of Using Generative AI':'% Value Of Using Generative AI'},
                color='Province')

fig.show()

fig.write_image("../visualization/Top5ProvincesAlreadyUsingGenerativeAI.pdf")







+------------------------------------------------+-------------------------------+----------------------------------+
|Business characteristics                        |Business's use of Generative AI|AverageValueOfUsingGenerativeAI(%)|
+------------------------------------------------+-------------------------------+----------------------------------+
|Information and cultural industries             |Already using Generative AI    |15.484615384615383                |
|Professional, scientific and technical services |Already using Generative AI    |14.99230769230769                 |
|Finance and insurance                           |Already using Generative AI    |12.453846153846154                |
|Arts, entertainment and recreation              |Already using Generative AI    |9.253846153846153                 |
|Manufacturing                                   |Already using Generative AI    |7.346153846153846                 |
+------------------------------------------------+------

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



+--------------------+-------------------------------+----------------------------------+
|GEO                 |Business's use of Generative AI|AverageValueOfUsingGenerativeAI(%)|
+--------------------+-------------------------------+----------------------------------+
|Ontario             |Already using Generative AI    |9.31063829787234                  |
|Prince Edward Island|Already using Generative AI    |7.278723404255319                 |
|British Columbia    |Already using Generative AI    |7.221276595744678                 |
|Nova Scotia         |Already using Generative AI    |7.097872340425534                 |
|Alberta             |Already using Generative AI    |6.448936170212766                 |
+--------------------+-------------------------------+----------------------------------+



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



2. Use of generative AI or planning to use generative AI has  helped in Accelerate development of creative content or Increase automation in tasks, without reducing employment.

In [ ]:
import plotly.express as px

from pyspark.sql.functions import col

# Define the values to include
values_to_include = ["Accelerate development of creative content", "Increase automation in tasks, without reducing employment"]

# Filter the data
filtered_df = cleaned_df.filter(col("Value or potential value created by Generative AI").isin(values_to_include))

#Caluculate the average value of using generative AI
filtered_df = filtered_df.groupBy("Business characteristics", "Value or potential value created by Generative AI") \
                .agg(F.avg("ValuePrdoucedByGenerativeAI(%)") \
                .alias("AverageValuePrdoucedByGenerativeAI(%)")) \
                .sort("AverageValuePrdoucedByGenerativeAI(%)", ascending=False) 

# Create a bar chart
filtered_df_pd = filtered_df.toPandas()

filtered_df_pd = filtered_df_pd.rename(columns={'Business characteristics': 'Sectors',
                                                'Value or potential value created by Generative AI': 'Value Created By Generative AI',
                                                'AverageValuePrdoucedByGenerativeAI(%)': 'Average Value Prdouced By Generative AI (%)'})

fig = px.bar(filtered_df_pd,
                x='Sectors',
                y='Average Value Prdouced By Generative AI (%)',
                title='Business Characteristics that Accelerate Development of Creative Content and Increase Automation in Tasks',
                labels={'Average Value Prdouced By Generative AI (%)':'% Value Prdouced By Generative AI'},
                color='Value Created By Generative AI')

fig.show()
fig.write_image("../visualization/BusinessCharacteristicsAccelerateDevelopmentOfCreativeContentAndIncreaseAutomationInTasks.pdf")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [ ]:
# Information and cultural industries 
# Professional, scientific and technical services
# Finance and insurance
# Arts, entertainment and recreation
# Manufacturing 

import plotly.express as px

from pyspark.sql.functions import col

# Define the values to include
values_to_include = ["Accelerate development of creative content", "Increase automation in tasks, without reducing employment"]

# Filter the data
filtered_df = cleaned_df.filter(col("Value or potential value created by Generative AI").isin(values_to_include))



values_to_include = ["Information and cultural industries ", "Professional, scientific and technical services ", "Finance and insurance ", "Arts, entertainment and recreation ", "Manufacturing "]

print(values_to_include)

filtered_df = filtered_df.filter(col("Business characteristics").isin(values_to_include))

filtered_df.show(truncate=False)

# #Calculate the average value of using generative AI
filtered_df = filtered_df.groupBy("Business characteristics", "Value or potential value created by Generative AI") \
                .agg(F.avg("ValuePrdoucedByGenerativeAI(%)") \
                .alias("AverageValuePrdoucedByGenerativeAI(%)")) \
                .sort("AverageValuePrdoucedByGenerativeAI(%)", ascending=False) 


# filtered_df.show(truncate=False)

# Create a bar chart
filtered_df_pd = filtered_df.toPandas()

filtered_df_pd = filtered_df_pd.rename(columns={'Business characteristics': 'Sectors',
                                                'Value or potential value created by Generative AI': 'Value Created By Generative AI',
                                                'AverageValuePrdoucedByGenerativeAI(%)': 'Average Value Prdouced By Generative AI (%)'})

fig = px.bar(filtered_df_pd,
                x='Sectors',
                y='Average Value Prdouced By Generative AI (%)',
                title='Business Characteristics that Accelerate Development of Creative Content and Increase Automation in Tasks',
                labels={'Average Value Prdouced By Generative AI (%)':'% Value Prdouced By Generative AI'},
                color='Value Created By Generative AI')

fig.show()

fig.write_image("../visualization/Top5BusinessCharacteristicsAccelerateDevelopmentOfCreativeContentAndIncreaseAutomationInTasks.pdf")


NameError: name 'cleaned_df' is not defined

In [ ]:
# Fitler by Business's use of Generative AI Have not considered using Generative AI and ValuePrdoucedByGenerativeAI(%) is 0 
# 

# transportation and warehousing
# construction
# agriculture, forestry, fishing and hunting 
# Mining, quarrying, and oil and gas extraction
# Wholesale trade

# filter only these values transportation and warehousing
# construction
# agriculture, forestry, fishing and hunting 
# Mining, quarrying, and oil and gas extraction
# Wholesale trade

#draw the comparision chart ValueOfUsingGenerativeAI and ValuePrdoucedByGenerativeAI for the above sectors

import plotly.express as px

from pyspark.sql.functions import col

# Define the values to include

values_to_include = ["Transportation and warehousing ", "Construction ", "Agriculture, forestry, fishing and hunting ", "Mining, quarrying, and oil and gas extraction ", "Wholesale trade "]

# Filter the data

filtered_df = cleaned_df.filter(col("Business characteristics").isin(values_to_include))

filtered_df = filtered_df.filter((col("Business's use of Generative AI") == "Have not considered using Generative AI") & (col("ValuePrdoucedByGenerativeAI(%)") == 0))

#consider average value of ValueOfUsingGenerativeAI(%) for the above sectors and show the comparision chart for the above sectors
filtered_df = filtered_df.groupBy("Business characteristics", "Business's use of Generative AI") \
                .agg(F.avg("ValueOfUsingGenerativeAI(%)") \
                .alias("AverageValueOfUsingGenerativeAI(%)")) \
                .sort("AverageValueOfUsingGenerativeAI(%)", ascending=False)

filtered_df.toPandas().to_csv("demo.csv")


# create a table using px  top 5 to show the average value of ValuePrdoucedByGenerativeAI(%) == 0 for the above sectors

filtered_df.show(truncate=False)



# Create a bar chart
filtered_df_pd = filtered_df.toPandas()

filtered_df_pd = filtered_df_pd.rename(columns={'Business characteristics': 'Sectors',
                                                
                                                'AverageValueOfUsingGenerativeAI(%)': 'Average Value Of Using Generative AI (%)'})

fig = px.bar(filtered_df_pd,
                x='Sectors',
                y='Average Value Of Using Generative AI (%)',
                title='Business Characteristics that Have not considered using Generative AI',
                labels={'Average Value Of Using Generative AI (%)':'% Potential Value Of Using Generative AI'},
                color='Sectors')

fig.show()
fig.write_image("../visualization/Top5BusinessCharacteristicsHaveNotConsideredUsingGenerativeAI.pdf")


NameError: name 'cleaned_df' is not defined